In [ ]:
n = 7 
P = np.array([0.5 for i in range(n)])
Q = np.array([0.2 for i in range(n)])
S = np.array([complex(P[i], Q[i]) for i in range(n)])
z = complex(0.02,0.05)*np.ones((n,n)) # impedances, get from feeder network object
lb_mode = True

# guess Vest as a value proportional to distance from substaion.
zmax = 10 # longest impedance to substation, in reality you'd compute it using impedance path function
zpath4 = 7 
zpath6 = 10
zpath7 = 9 # impedance path to each edge node of nwk
Vest = {}
Vest[4] = 1 - (0.2 * (zpath4 / zmax)) # per unit
Vest[6] = 1 - (0.2 * (zpath6 / zmax))
Vest[7] = 1 - (0.2 * (zpath7 / zmax))

# When looking at computing Plb or Qlb, actuator worst case scenario is
# netative the capacity value. WHen looking at computing
computing_lb = True # pass computing_lb argument in, so that you can say soemthing like 

Sact = np.zeros(n)
if lb_mode == True: 
    Sact[4] = -.3 # per unit, actuator capacity value, at node 4, .3 pu ==> 500kw wrt 5000/3 kVA
    Sact[5] = -.3 # capacity value, at node 5
else:
    Sact[4] = .3
    Sact[5] = .3

# For example comput impedance between node and subst, and let zmax be
# longest path to substation
V = 1 - (z[0] / zmax)
I = np.zeros((n,n)) 
I = I.astype('complex128')
Sloss = np.zeros((n,n)) # will populate 
Sloss = Sloss.astype('complex128')
# -----------------------------------------
# Compute line losses below
# -----------------------------------------
# Line loss of line 34, edge line
i = (S[3] + Sact[4]) / Vest[4]
I[2][3] = complex(i.real, (-1) * i.imag) # take complex conjugate, Sact1 is at node 4
Sloss[2][3] = ((I[2][3])**2) * z[2][3]

# Line loss of line23, non-edge line
V[2] = V[3] + I[2][3] * z[2][3]
i = (S[2] / V[2]) 
real, imag = i.real, (-1) * i.imag
I[1][2] = complex(real, imag) + I[2][3] # I30+I(3,4)
Sloss[1][2] = (I[1][2]**2) * z[1][2]

# Notice node 2 is a fork, so we've finished finding losses in branch 2-4

# Line loss of line 56, edge line
i = S[5]/Vest[6]
I[4][5] = complex(i.real, (-1) * i.imag) #take complex conjugate
Sloss[4][5] = (I[4][5]**2) * z[4][5]

# Notice node 5 is a fork, so we've finished finding losses in branch 5-6

# Line loss of line 57, edge line
i = S[6]/Vest[7]
I[4][6] = complex(i.real, (-1) * i.imag) #take complex conjugate
Sloss[4][6] = (I[4][6]**2) * z[4][6]

# Notice node 5 is a fork, so we've finished finding losses in branch 5-7

# Notice among the forks, fork 5 is further downstream than fork 2 so
# resolve fork at node 5 first

# Line loss of line25, line connected to fork at node 5
Vfork1 = V[5] + I[4][5] * z[4][5]
Vfork2 = V[6] + I[4][6] * z[4][6]
checkVclose = ((Vfork1 - Vfork2) / Vfork1) < 0.1 # check not more than 10% different
if checkVclose == False:
    print('greater than 10% difference ==> choose different edge V') # dont need to implement this error call in python, just manually check that they are close

V[4] = np.mean([Vfork1, Vfork2]) # average of all Vfork estimates, one for each branch of the fork
i = (S[4] + Sact[5]) / V[4]
I[1][4] = I[4][5] + I[4][6] + complex(i.real, (-1) * i.imag) # Sact2 is at node 5
Sloss[1][2] = (I[1][4]**2) * z[1][4]

# Line loss of line12, line connected to fork at node 2
Vfork1 = V[4] + (I[1][4] * z[1][4])
Vfork2 = V[2] + I[1][2] * z[1][2]
checkVclose = ((Vfork1 - Vfork2) / Vfork1) < 0.1 # check not more than 10% different
if checkVclose == False:
    print('greater than 10% difference ==> choose different edge V');

V[1] = np.mean([Vfork1, Vfork2]) # average of all Vfork estimates, one for each branch of the fork
i = S[1]/V[1]
I[0][1] = I[1][2] + I[1][4] + complex(i.real, (-1) * i.imag)
Sloss[0][1] = (I[0][1]**2) * z[0][1]

# Line loss of line01, non-edge line
V[0] = V[1] + I[0][1] * z[0][1]
i = (S[2]/V[2]) 
I[1][2] = complex(i.real, (-1) * i.imag) + I[2][3]# I30+I(3,4)
Sloss[1][2] = (I[1][2]**2) * z[1][2]

# Now sum everything to get Seq
idxDown = [3, 4, 5, 6, 7] # indices of nodes downstream from perf_node
Seq = sum(sum(Sloss)) + sum(S) + sum(Sact)
Peq = Seq.real
Qeq = Seq.imag
Peq, Qeq, Seq